In [2]:
import psycopg2
import pandas as pd
from datetime import date
from datetime import timedelta

In [3]:
conn = psycopg2.connect(
    host="mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com",
    database="mid_term_project",
    user="lhl_student",
    password="lhl_student")

def query(query):
    return pd.read_sql(query, conn)

In [7]:
query(""" SELECT * FROM flights
        LIMIT 10
""").columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name'],
      dtype='object')

In [23]:
flights = query("""SELECT * FROM flights
        LIMIT 1000000""")

In [26]:
flights.to_csv('flights.csv')

In [20]:
flights_random_sample = flights.sample(n=100)

In [21]:
flights_random_sample

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
38771,2019-01-22,UA,UA_CODESHARE,UA,3682,YX,N855RW,3682,12339,IND,...,977.0,0.0,0.0,7.0,0.0,47.0,None,NaN,NaN,None
14639,2019-01-20,UA,UA_CODESHARE,UA,3652,YX,N855RW,3652,14730,SDF,...,788.0,0.0,0.0,17.0,0.0,0.0,None,NaN,NaN,None
808,2019-01-20,WN,WN,WN,4597,WN,N8708Q,4597,12889,LAS,...,414.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
76883,2019-01-23,DL,DL,DL,2172,DL,N717JL,2172,13487,MSP,...,223.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
86624,2019-01-24,DL,DL,DL,2043,DL,N935AT,2043,10397,ATL,...,341.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15246,2019-01-21,B6,B6,B6,412,B6,N184JB,412,13930,ORD,...,867.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
40933,2019-01-22,DL,DL,DL,1538,DL,N819DN,1538,14679,SAN,...,1532.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
33160,2019-01-21,DL,DL_CODESHARE,DL,3387,9E,N834AY,3387,14524,RIC,...,288.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
47024,2019-01-22,WN,WN,WN,394,WN,N8555Z,394,11292,DEN,...,909.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
